# Audio classification with a pipeline

Ses sınıflandırması, içeriğine göre bir ses kaydına bir veya daha fazla etiket atanmasını içerir. Bu etiketler müzik, konuşma veya gürültü gibi farklı ses kategorilerine ya da kuş sesi veya araba motoru sesi gibi daha spesifik kategorilere karşılık gelebilir.

En popüler ses dönüştürücülerinin nasıl çalıştığına dair ayrıntılara girmeden ve özel bir modele ince ayar yapmadan önce, Transformers ile yalnızca birkaç satır kodla ses sınıflandırması için kullanıma hazır önceden eğitilmiş bir modeli nasıl kullanabileceğinizi görelim.

Devam edelim ve önceki ünitede keşfettiğiniz MmINDS-14 veri kümesini kullanalım. Hatırlarsanız, MINDS-14 bir e-bankacılık sistemine çeşitli dil ve lehçelerde sorular soran kişilerin kayıtlarını içeriyor ve her kayıt için intent_class değerine sahip. Kayıtları aramanın amacına göre sınıflandırabiliriz.

Daha önce olduğu gibi, boru hattını denemek için verilerin en-AU alt kümesini yükleyerek başlayalım ve çoğu konuşma modelinin gerektirdiği 16kHz örnekleme oranına yükseltelim.

In [1]:
from datasets import load_dataset, Audio

minds = load_dataset(
    "PolyAI/minds14", name="en-AU", 
    split="train", trust_remote_code=True,
)
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 654
})

Bir ses kaydını bir dizi sınıfa ayırmak için Transformers'ın ses sınıflandırma pipeline'ını kullanabiliriz. Bizim durumumuzda, amaç sınıflandırması için ve özellikle MINDS-14 veri kümesi üzerinde ince ayar yapılmış bir modele ihtiyacımız var. Neyse ki Hub'da tam da bunu yapan bir model var! Pipeline() fonksiyonunu kullanarak yükleyelim:

In [2]:
from transformers import pipeline

classifier = pipeline(
    task="audio-classification",
    model="anton-l/xtreme_s_xlsr_300m_minds14",
)

2024-08-08 06:37:34.421611: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 06:37:34.421838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 06:37:34.575856: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at anton-l/xtreme_s_xlsr_300m_minds14 were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model tha

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

In [3]:
example = minds[0]

In [4]:
classifier(example["audio"]["array"])

[{'score': 0.9625310301780701, 'label': 'pay_bill'},
 {'score': 0.02867276780307293, 'label': 'freeze'},
 {'score': 0.003349797334522009, 'label': 'card_issues'},
 {'score': 0.002005803631618619, 'label': 'abroad'},
 {'score': 0.000848432769998908, 'label': 'high_value_payment'}]

In [5]:
id2label = minds.features["intent_class"].int2str
id2label(example["intent_class"])

'pay_bill'